# Dependancy Installations

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install faiss-gpu

In [ ]:
import faiss,numpy

In [ ]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('stsb-roberta-base-v2')

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import spacy
import nltk
import re

In [ ]:
import pandas as pd

# Setting Variables

In [ ]:
job_file_path='/content/Jobs.csv'
candidate_file_path='/content/Candidate.csv'

# Data Loading

In [ ]:
jobs_df=pd.read_csv(job_file_path,encoding = "ISO-8859-1")
cand_df=pd.read_csv(candidate_file_path,encoding = "ISO-8859-1")

In [ ]:
del jobs_df['Date']                   # Since contained all NAN values
del jobs_df['Job ID']                 # Since contained all NAN values
# jobs_df=jobs_df.dropna()              # Dropping further NAN values
jobs_df.fillna('NON', inplace=True)

In [ ]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003 entries, 0 to 2002
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company Name      2003 non-null   object
 1   Post              2003 non-null   object
 2   Location          2003 non-null   object
 3   Description       2003 non-null   object
 4   Level             2003 non-null   object
 5   Type              2003 non-null   object
 6   Function          2003 non-null   object
 7   Industry          2003 non-null   object
 8   Responsibilities  2003 non-null   object
 9   Qualifications    2003 non-null   object
dtypes: object(10)
memory usage: 156.6+ KB


In [ ]:
jobs_df.shape

(2003, 10)

In [ ]:
# Relevant columns for job dataframe
job_responsibility_column=8
job_description_column=3
job_qualification_column=9

In [ ]:
cand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16201 entries, 0 to 16200
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               16198 non-null  float64
 1   About            16084 non-null  object 
 2   Accomplishments  11881 non-null  object 
 3   Education        15928 non-null  object 
 4   Experience       15828 non-null  object 
 5   Industry         27 non-null     object 
 6   Level            16198 non-null  object 
 7   Location         16198 non-null  object 
 8   Skills           15886 non-null  object 
 9   Type             16198 non-null  object 
 10  Certification    8023 non-null   object 
dtypes: float64(1), object(10)
memory usage: 1.4+ MB


In [ ]:
cand_df.shape

(16201, 11)

In [ ]:
# cand_df.iloc[2]

In [ ]:
# cand_df.iloc[1]

In [ ]:
# Relevant columns for candidate dataframe
candidate_experience_column=4
candidate_education_column=3
candidate_skills_column=8
candidate_certificates_column=10

In [ ]:
jobs_df=jobs_df.drop_duplicates(keep='first',subset=['Description','Responsibilities'])    # Dropping duplicates

In [ ]:
cand_df=cand_df.drop_duplicates(keep='first',subset=['Education','Skills','Experience'])      # Dropping duplicates

In [ ]:
jobs_df.shape

(197, 10)

In [ ]:
cand_df.shape

(1488, 11)

In [ ]:
cand_df.to_csv('cand_df.csv')

In [ ]:
cand_df.iloc[355]

ID                                                               496
About                                          \n\n    About\n  \n\n
Accomplishments                 Database Management Systems & Design
Education          \n\n \n\n\n\n\nUniversity of St. Thomas\n\nDeg...
Experience           \n\n \n\n\n\nAssistant Manager, Digital Chan...
Industry                                                         NaN
Level                                                    Entry level
Location                                                      Riyadh
Skills             \n\n\n\n\n \n            Research\n          \...
Type                                                       Full-time
Certification      \n\n \n\n\n\n\n    Graduate Certificate in Bus...
Name: 496, dtype: object

In [ ]:
# Concatentaing text from desired column i.e description,responsibility,and qualification
job_text=[]
job_ids=[]
for i in range(len(jobs_df)):
  concatenated_text=(jobs_df.iloc[i,job_description_column]+ '. '+ jobs_df.iloc[i,job_responsibility_column]+'. '+ jobs_df.iloc[i,job_qualification_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  job_text.append(text)
  job_ids.append(i)


# Concatentaing text from desired column i.e experience, education,skills and certficates
candidate_text=[]
cand_ids=[]
for i in range(len(cand_df)):
  concatenated_text=str(cand_df.iloc[i,candidate_experience_column])+ '. '+str(cand_df.iloc[i,candidate_education_column])+ \
  '. '+str(cand_df.iloc[i,candidate_skills_column])+'. '+str(cand_df.iloc[i,candidate_certificates_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  text=text.replace('\n','')
  candidate_text.append(text)
  cand_ids.append(i)
jobs_df['job_text']=job_text
#jobs_df['job_text']=job_text
cand_df['candidate_text']=candidate_text
#cand_df['job_text']=job_text

In [ ]:
candidate_text[0]

"7+ years Minimum as PMO. Bachelor's degree in Computer Science. Experience to run medium to large projects. I have good knowledge of project financials.Expert in all PMO functions, data/records managements, reportingStrong in planning and tracking.fluent in English and Arabic language. nan"

In [ ]:
job_text[0]

"As a Senior Program Manager you work with our team to facilitate decision making, judgment, resolving constraints and enable high quality on program execution and delivery.. Manage technology programs, defining milestones and success criteria, resource allocation and successful on-time delivery.. Exercise technical judgment in working with large, cross-functional teams.. Develop tools and processes to improve software engineering productivity.. Communicate schedules, priorities, and status to all levels.. Proven work experience program managing datawatehouse systems. Strong knowledge of software development life cycle methodologies, tools and processes. Demonstrated ability or preference to work across multiple teams.. Experience in writing clear, concise and comprehensive program plans and schedules. Hands-on experience with Prioritization and issue management. Proven record on facilitating decision making and judgment. Experience working in an Agile/Scrum development process    Bach

In [ ]:
# Creating set to handle duplicates, since data has lot of duplicates
complete_text=job_text+candidate_text

In [ ]:
# Create mapping to map back to original dataframe
indexes=[i for i in range(len(complete_text))]
mapping=dict(zip(indexes,(job_ids+cand_ids)))

In [ ]:
mapping[202]

5

In [ ]:
complete_text[214]

'  Performance and development managerCompany Name      Afkar alQimam        Full-timeDates EmployedMar 2019 ï¿½ï¿½ï¿½ PresentEmployment Duration2 yrs 5 mosLocationriyadh    All HR functions  .  University of OxfordDegree NameDiplomaField Of StudyHuman Resources Management/Personnel Administration, General Dates attended or expected graduation2013 ï¿½ï¿½ï¿½ 2013.              Recruiting            See 114 endorsements for Recruiting99+      Endorsed by Khalid Aldosari   - ChHRC and 3 others who are highly skilled at this       Endorsed by 3 of Mohammedï¿½ï¿½ï¿½s colleagues at ALJOMAIH BOTTLING PLANTS. nan'

In [ ]:
cand_df.iloc[i,11]

'nan. nan. nan. nan'

In [ ]:
# code to extract number of professional years from candidate experience
import re
pattern=re.compile(r'\d [y]r')
years=[]
for i in range(len(cand_df)):
  try:
    years.append(pattern.findall(cand_df.iloc[i,11])[0])
  except:
    years.append('NA')
cand_df['Duration']=years

In [ ]:
cand_df

,ID,About,Accomplishments,Education,Experience,Industry,Level,Location,Skills,Type,Certification,candidate_text,Duration
0,0.0,NaN,NaN,Bachelor's degree in Computer Science,7+ years Minimum as PMO,IT,Senior,Riyadh,Experience to run medium to large projects. I ...,Full-time,NaN,7+ years Minimum as PMO. Bachelor's degree in ...,NA
1,1.0,NaN,NaN,Bachelor's degree in Computer Science,My experience is in program management and pro...,IT,Senior,Riyadh,influential member of a highly integrated team...,Full-time,NaN,My experience is in program management and pro...,NA
2,2.0,NaN,NaN,Bachelor's degree in Computer Science. Banking...,In my experience I Manageed technology program...,IT,Senior,Riyadh,Yes I have the Ability to exercise technical j...,Full-time,NaN,In my experience I Manageed technology program...,NA
3,3.0,NaN,NaN,Bachelor's degree in information system\nExper...,Experience in delivery of the business analysi...,IT,entry level,Riyadh,I have the Ability to exercise technical judgm...,Full-time,NaN,Experience in delivery of the business analysi...,NA
4,4.0,NaN,NaN,Bachelor's degree in Computer Science\nExcelle...,"working with large, cross-functional teams.\nD...",IT,Senior,Riyadh,experience in program managing datawatehouse s...,Full-time,NaN,"working with large, cross-functional teams.Dev...",NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16194,16194.0,NaN,NaN,Bachelor of Science in Nursing.\ncurrent Nursi...,I have 7.5 years of icu experience (cardiovasc...,healthcare Nursing,Entry level,Riyadh,Strong organization and people management skil...,Full-time,NaN,I have 7.5 years of icu experience (cardiovasc...,NA
16195,16195.0,NaN,NaN,Bachelor of Science in Nursing.\ncurrent Nursi...,"Highly motivated, self-starter, hard-working, ...",healthcare Nursing,Entry level,Riyadh,I have good oral and written communication and...,Full-time,NaN,"Highly motivated, self-starter, hard-working, ...",NA
16196,16196.0,NaN,NaN,"Registered Nurse, with PRC licensed \nProfessi...",More than 5 years experienced as a staff nurse...,healthcare Nursing,Entry level,Riyadh,with knowledge and clinical assessment skills ...,Full-time,NaN,More than 5 years experienced as a staff nurse...,NA
16197,16197.0,NaN,NaN,Bachelor of Science in Nursing\nEnglish Langua...,"Pursuing a highly rewarding career, seeking a ...",healthcare Nursing,Entry level,Riyadh,Successful treatment of Hemodialysis both Acut...,Full-time,NaN,"Pursuing a highly rewarding career, seeking a ...",NA


# Data Cleaning

In [ ]:
# Data preprocessing function
def preprocess(sentence):
  sentence = "".join(filter(lambda x: not x.isdigit(), sentence))
  sentence = ' '.join([i for i in sentence.split() if '@' not in i])
  sentence=re.sub(' +', ' ', sentence)
  sentence=' '.join([item for item in sentence.split() if '@' not in item])
  sentence = ' '.join([item for item in sentence.split() if len(item)>2])
  sentence=re.sub("[\(\[]*?[\)\]]", "", sentence)
  sentence = re.sub(r"[^a-zA-Z0-9.,]+", ' ', sentence)
  sentence=sentence.rstrip("\n")
  sentence=sentence.strip()
  strText=sentence.split()
  sentence=" ".join(strText)
  sentence=sentence.lower()
  return sentence

In [ ]:
# Creating list of cleaned text, using above preprocessing function
cleaned_complete=[]
for i in range(len(complete_text)):
  cleaned_complete.append(preprocess(complete_text[i]))

In [ ]:
len(set(complete_text))

1685

In [ ]:
# Setting Faiss variables
d = 768                   # dimension of vectors
nb = len(set(complete_text))              # database size (jobs+candidates)
nq = 1                    # nb of queries
k = len(set(complete_text))             # To see 1000 nearest neighbors

In [ ]:
n_jobs=10                 # number of similar jobs to be retrieved
n_candidates=10           # number of similar candidates to be retrieved

# Generating embeddings and FAISS indexing

In [ ]:

# Creating embeddings 
cleaned_embed=embed_model.encode(cleaned_complete,show_progress_bar=True)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

In [ ]:
cleaned_embed

array([[ 1.1814553e-01,  9.7288615e-01,  3.3469856e-01, ...,
         1.2492381e-01, -1.0351658e-01, -1.0842588e+00],
       [-7.3922265e-01,  5.6362104e-01,  3.3563229e-01, ...,
        -1.4706001e-01,  6.1759800e-01, -7.1749043e-01],
       [-2.9773973e-02, -4.9499577e-01, -3.1987113e-01, ...,
        -1.5453414e+00,  4.2558202e-01, -8.2937914e-01],
       ...,
       [ 2.6517290e-01,  3.7177280e-01,  7.5391963e-02, ...,
        -8.6359924e-01,  8.1428039e-01, -1.1651595e+00],
       [ 7.0976324e-02,  3.3122394e-02,  2.9116493e-01, ...,
        -1.1554957e-01,  1.3034269e+00, -3.0651158e-01],
       [ 5.7604343e-01,  9.3502933e-01,  4.9262270e-01, ...,
         2.0542111e-01,  4.2908871e-01, -2.0061723e-04]], dtype=float32)

In [ ]:
import numpy as np
np.random.seed(1234)             # make reproducible
res = faiss.StandardGpuResources()  # use a single GPU
index_flat = faiss.IndexFlatL2(d)  # build a flat (CPU) index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index_flat.add(cleaned_embed)         # add vectors to the index
print(gpu_index_flat.ntotal)

1685


In [ ]:
gpu_index_flat

<faiss.swigfaiss.GpuIndexFlat; proxy of <Swig Object of type 'faiss::gpu::GpuIndexFlat *' at 0x7f83f62bbde0> >

In [ ]:
len(set(job_text))

197

In [ ]:
len(set(candidate_text))

1488

In [ ]:
len(set(complete_text))

1685

In [ ]:
# Creating distance and index array, here k refers to number of nearest neighbors to be searched
D, I = gpu_index_flat.search(cleaned_embed, k) 

In [ ]:
act=[]
for ele in I[1]:
  if ele >= len(set(job_text)):
    act.append(ele)
act[:10]


[1677, 1682, 587, 674, 1308, 1207, 957, 1680, 553, 810]

In [ ]:
for F in act[:10]:
   print(F)
   print('Mapping')
   print(mapping[F])
   print('**************')
   print(complete_text[F])
   print('**************')


1677
Mapping
1480
**************
I have two years experience in Emergency Room Nurse with a demonstrated history of working in the medical practice industry. I Provide and direct patient care utilising the nursing process within established hospital, divisional and unit specific policies and procedures. Skilled in Healthcare, Clinical Research, Strategic Planning, Microsoft Office, and Nursing. I have an experience in clinical assessment skills appropriate to clinical area Takes direct and appropriate action in urgent and emergency situations. I can assets pertinent information in the patients healthcare record in a timely and ongoing manner. . Strong healthcare services professional with a Master of Arts - MA focused in Registered Nursing/Registered Nurse and Bachelor of Science in Nursing.current Nursing license English and Arabic Language Skills with Computer Skills. High oral and written communication and collaboration skills when interacting with patients/families and healthcare

In [ ]:
mapping[1677]

1480

# Extracting similar candidates for a given job

In [ ]:
'''
Extracts 10 most similar candidates for a given job along with distance scores
'''
ids=[]
retrieved_candidates=[]
distance=[]
cand_index=[]
job=[]
for i in range(len(set(job_text))):
  act=[]
  dist=[]
  ids=[]
  for ele,el in (zip(I[i],D[i])):
    if ele>= len(set(job_text)):
      act.append(complete_text[ele])
      dist.append(el)
      ids.append(ele)
  job.append(complete_text[i])
  retrieved_candidates.append(act[:10])
  distance.append(dist[:10])
  cand_index.append(ids[:10])


In [ ]:
jobs_dataframe=pd.DataFrame(list(zip(job,cand_index,retrieved_candidates,distance)),columns=['Job Description','Candidate ID','Retrieved Candidates','Distance Score'])

In [ ]:
jobs_dataframe

,Job Description,Candidate ID,Retrieved Candidates,Distance Score
0,As a Senior Program Manager you work with our ...,"[203, 206, 205, 204, 202, 909, 199, 201, 552, ...",[program manager leading team to facilitate de...,"[63.20749, 67.35776, 93.85739, 104.94826, 107...."
1,NON. NON. NON,"[365, 765, 520, 503, 761, 259, 495, 280, 940, ...",[ UnemployedCompany Name none Dates Emp...,"[409.41748, 426.85437, 430.76672, 433.06274, 4..."
2,The Data Scientist is responsible for expandin...,"[207, 913, 203, 683, 205, 204, 1255, 1501, 752...",[A Data Scientist responsible for expanding an...,"[81.14218, 189.78131, 196.14465, 198.76317, 19..."
3,Ensure delivering on UTR targets and active ri...,"[683, 1545, 909, 1057, 1254, 338, 1220, 1214, ...",[ Company NameLeading Machine Learning FirmTo...,"[205.61691, 212.39093, 224.46071, 227.99146, 2..."
4,. To coordinate and collaborate with various d...,"[202, 647, 203, 1575, 715, 206, 507, 909, 1561...",[Work with stakeholders throughout the organiz...,"[176.41519, 196.55205, 205.06476, 205.25043, 2..."
...,...,...,...,...
192,We are seeking a Data Scientist to work with D...,"[207, 518, 1059, 672, 1127, 752, 1154, 1149, 8...",[A Data Scientist responsible for expanding an...,"[155.78656, 188.75848, 192.60553, 206.82674, 2..."
193,"The Data Scientist will support our pricing, s...","[207, 512, 1245, 913, 683, 344, 200, 388, 1369...",[A Data Scientist responsible for expanding an...,"[105.03473, 158.0285, 177.71902, 178.54065, 18..."
194,"Since 2011, General Assembly has transformed t...","[630, 1377, 464, 1252, 1624, 1119, 1391, 427, ...",[ Jeel Tamooh MemberCompany Name Boston...,"[150.2782, 153.34793, 153.74188, 154.11832, 16..."
195,Master Works is one of the leading companies i...,"[683, 512, 1431, 207, 785, 787, 736, 1149, 464...",[ Company NameLeading Machine Learning FirmTo...,"[172.55867, 175.3205, 179.12578, 180.7172, 183..."


In [ ]:
# jobs_dataframe

In [ ]:
jobs_dataframe.iloc[0,0]

"As a Senior Program Manager you work with our team to facilitate decision making, judgment, resolving constraints and enable high quality on program execution and delivery.. Manage technology programs, defining milestones and success criteria, resource allocation and successful on-time delivery.. Exercise technical judgment in working with large, cross-functional teams.. Develop tools and processes to improve software engineering productivity.. Communicate schedules, priorities, and status to all levels.. Proven work experience program managing datawatehouse systems. Strong knowledge of software development life cycle methodologies, tools and processes. Demonstrated ability or preference to work across multiple teams.. Experience in writing clear, concise and comprehensive program plans and schedules. Hands-on experience with Prioritization and issue management. Proven record on facilitating decision making and judgment. Experience working in an Agile/Scrum development process    Bach

In [ ]:
# Writing output dataframe as json file
jobs_dataframe.to_csv('job_dataframe.csv')

# Extracting similar jobs for a given candidate

In [ ]:
retrieved_jobs=[]
job_index=[]
distance=[]
candidates=[]
for i in range(len(set(job_text)),len(complete_text)):
  act=[]
  dist=[]
  ids=[]
  for ele,el in (zip(I[i],D[i])):
    if ele < len(set(job_text)):
      act.append(complete_text[ele])
      dist.append(el)
      ids.append(ele)
  candidates.append(complete_text[i])
  retrieved_jobs.append(act[:10])
  distance.append(dist[:10])
  job_index.append(ids[:10])

In [ ]:
act=[]
for ele in I[195]:
  if ele < len(set(job_text)):
    act.append(ele)
act[:10]

[195, 193, 196, 88, 162, 123, 192, 16, 87, 34]

In [ ]:
complete_text[195]

"Master Works is one of the leading companies in our field in the area. We are hiring a data scientist specialist to join our team in Saudi Arabia, specifically in Riyadh. If you're excited to be part of a winning team, Master Works is a great place to grow your career. . Design and develop statistical & machine learning models to support business decisions and help improve products and services.. Design and develop data pipelines with the right tools, databases and ETL process.. Design and develop user-friendly platforms to enable easy testing of ML pipelines, simulate data and visualize performance metrics for evaluation.. Identify and discover hidden patterns in complex structured and unstructured data sets.. Help in implementing data analytics and data governance roadmaps to help drive consistency in data sources and different data flows.. Experience with Media related data and KPIs is an asset.. Saudi Nationality.. . ï¿½ Proficiency in Python, SQL and experience with ML libraries 

In [ ]:
cleaned_complete[0]

'senior program manager you work with our team facilitate decision making, judgment, resolving constraints and enable high quality program execution and delivery.. manage technology programs, defining milestones and success criteria, resource allocation and successful on time delivery.. exercise technical judgment working with large, cross functional teams.. develop tools and processes improve software engineering productivity.. communicate schedules, priorities, and status all levels.. proven work experience program managing datawatehouse systems. strong knowledge software development life cycle methodologies, tools and processes. demonstrated ability preference work across multiple teams.. experience writing clear, concise and comprehensive program plans and schedules. hands on experience with prioritization and issue management. proven record facilitating decision making and judgment. experience working agile scrum development process bachelor s degree computer science, equivalent p

In [ ]:
candidate_dataframe=pd.DataFrame(list(zip(candidates,job_index,retrieved_jobs,distance)),columns=['Candidate Description','Job Ids','Retrieved Jobs','Distance Score'])

In [ ]:
# candidate_dataframe=pd.DataFrame(list(zip(candidates,retrieved_jobs,distance)),columns=['Candidate Description','Retrieved Jobs','Distance Score'])

In [ ]:
# Writing output dataframe as json file
candidate_dataframe.to_csv('candidate_dataframe.csv')

In [ ]:
candidate_dataframe

,Candidate Description,Job Ids,Retrieved Jobs,Distance Score
0,7+ years Minimum as PMO. Bachelor's degree in ...,"[84, 138, 10, 164, 64, 189, 17, 127, 193, 33]",[manage and oversee the performance of the fie...,"[222.66568, 229.6702, 233.53961, 243.99777, 24..."
1,My experience is in program management and pro...,"[84, 138, 64, 10, 33, 0, 185, 34, 122, 38]",[manage and oversee the performance of the fie...,"[163.4848, 171.89131, 179.15918, 186.26108, 19..."
2,In my experience I Manageed technology program...,"[15, 14, 12, 33, 0, 84, 122, 189, 125, 138]",[To manage digital projects including planning...,"[159.7435, 159.7435, 164.33044, 166.2811, 167...."
3,Experience in delivery of the business analysi...,"[12, 138, 125, 193, 14, 15, 185, 34, 2, 10]",[. The jobholder is responsible for Developmen...,"[176.7414, 196.38264, 197.32306, 197.4284, 202..."
4,"working with large, cross-functional teams.Dev...","[33, 10, 0, 122, 84, 138, 185, 12, 34, 127]",[We are working hand in hand with your busines...,"[156.50769, 156.86874, 173.88095, 176.0954, 17..."
...,...,...,...,...
1471,SupperSystem administration and IT certificati...,"[189, 17, 164, 138, 18, 44, 194, 84, 64, 87]",[Application Support using Microsoft & Linux t...,"[254.22229, 260.83768, 263.36023, 271.6305, 27..."
1472,Supper 3 years of experience high profici...,"[64, 138, 38, 6, 10, 17, 84, 193, 34, 82]",[We are looking to employ an HR Assistant with...,"[186.932, 204.63496, 210.83298, 212.42706, 221..."
1473,Supper 3 years in a managerial role Strong rec...,"[64, 138, 38, 53, 84, 82, 10, 17, 100, 87]",[We are looking to employ an HR Assistant with...,"[156.87146, 183.48149, 187.556, 198.9774, 199...."
1474,Supper 1 to 3 years of experience working in ...,"[64, 138, 84, 18, 17, 189, 87, 38, 12, 44]",[We are looking to employ an HR Assistant with...,"[222.91348, 223.2253, 244.83813, 250.1925, 255..."


In [ ]:
# candidate_dataframe

In [ ]:
candidate_dataframe.to_excel('candidate_dataframe.xlsx')

In [ ]:
file1 = open("out.txt","w")
for i in range(len(jobs_dataframe)):
# \n is placed to indicate EOL (End of Line)
  file1.write(jobs_dataframe.iloc[i,0])
  file1.write('\n')
  for j in range(10):
    file1.write('Similar Candidate'+str(j))
    file1.write('\n')
    file1.write(jobs_dataframe.iloc[i,2][j])
    file1.write('\n')
  file1.write('*'*100) 
file1.close() #to change file access modes